In [27]:
# What version of Python do you have?
import sys

from collections import Counter

import tensorflow.keras
import pandas as pd
import sklearn as sk
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTENC
import tensorflow as tf
import seaborn as sns
import math
import matplotlib.pyplot as plt
import numpy as np
import re
import demoji
import datetime

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

Tensor Flow Version: 1.13.1
Keras Version: 2.2.4-tf

Python 3.6.10 |Anaconda, Inc.| (default, Jan  7 2020, 15:01:53) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Pandas 1.0.1
Scikit-Learn 0.22.1
GPU is NOT AVAILABLE


In [2]:
stock_data_df = pd.read_csv('GOOG.csv')

In [3]:
stock_data_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-02-06,1139.569946,1147.000000,1112.770020,1115.229980,1115.229980,2105600
1,2019-02-07,1104.160034,1104.839966,1086.000000,1098.709961,1098.709961,2044800
2,2019-02-08,1087.000000,1098.910034,1086.550049,1095.060059,1095.060059,1075800
3,2019-02-11,1096.949951,1105.944946,1092.859985,1095.010010,1095.010010,1065200
4,2019-02-12,1106.800049,1125.295044,1105.849976,1121.369995,1121.369995,1609100


In [4]:
dates = stock_data_df.loc[:, 'Date'].to_list()
close_prices = stock_data_df.loc[:, 'Adj Close'].to_list()
volumes = stock_data_df.loc[:, 'Volume'].to_list()

- stock price feature fields:
    - Date
    - Past N price
    - Past N - 1 price
    - ...
    - Past 1 price
    - Label

In [5]:
time_window = 2

In [6]:
def stock_price_feature_label_generator(time_window = 3):
    if time_window <= 0: 
        raise ValueError("argument time_window has to be at least 1")
    # use "time_window" previous days to predict the movement of the next day
    label = [-1] + [1 if close_prices[i] > close_prices[i - 1] else 0 for i in range(1, len(close_prices))]
    result = pd.DataFrame(dates[time_window: ], columns = ['Date'])
    result.insert(loc=result.shape[1], column="Label", value=label[time_window: ])
    for i in range(time_window):
        result.insert(loc=result.shape[1], column="Past " + str(time_window - i) + " Day Adj. Price", value=np.divide(close_prices[i: len(close_prices) - time_window + i], volumes[i: len(volumes) - time_window + i]))
#     for i in range(time_window):
#         result.insert(loc=result.shape[1], column="Past " + str(time_window - i) + " Day Price", value=close_prices[i: len(close_prices) - time_window + i])
#     for i in range(time_window):
#         result.insert(loc=result.shape[1], column="Past " + str(time_window - i) + " Volume", value=volumes[i: len(volumes) - time_window + i])
    return result

In [7]:
# stock_price_feature = stock_price_feature_label_generator(time_window)
# stock_price_feature.head()

In [9]:
only_date_and_label = stock_price_feature_label_generator(1).drop(columns=['Past 1 Day Adj. Price'])

In [10]:
only_date_and_label.head()

,Date,Label
0,2019-02-07,0
1,2019-02-08,0
2,2019-02-11,0
3,2019-02-12,1
4,2019-02-13,0


In [34]:
stock_price_feature = stock_data_df[['Date', 'Adj Close']]
stock_price_feature['Label'] = [-1] + only_date_and_label.Label.to_list()
stock_price_feature = stock_price_feature[['Date', 'Label', 'Adj Close']]
stock_price_feature.head()

/Users/frostace/.conda/envs/bert_env/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Date,Label,Adj Close
0,2019-02-06,-1,1115.229980
1,2019-02-07,0,1098.709961
2,2019-02-08,0,1095.060059
3,2019-02-11,0,1095.010010
4,2019-02-12,1,1121.369995


In [11]:
aggregated_vector = pd.read_csv("aggregated_vector.csv")

In [12]:
aggregated_vector.shape

(366, 801)

In [13]:
only_date_and_label.shape

(252, 2)

In [35]:
feature_with_label_date = stock_price_feature.merge(aggregated_vector, left_on="Date", right_on="Date")

In [38]:
feature_with_label_date.head()

,Date,Label,Adj Close,0,1,2,3,4,5,6,...,num_pos_VB,num_pos_NN,num_neg_JJ,num_neg_RB,num_neg_VB,num_neg_NN,total_score_JJ,total_score_RB,total_score_VB,total_score_NN
0,2019-02-06,-1,1115.229980,-31.065374,26.964167,207.587202,-29.100772,95.531243,-118.985815,-24.375886,...,3.998121,55.419917,53.802890,42.127785,0.0,59.779481,23.410036,4.487775,1.999061,9.431500
1,2019-02-07,0,1098.709961,-1.954738,-14.441650,63.646112,-10.731836,39.554150,-40.863590,-10.272327,...,2.000000,18.000000,37.000000,8.000000,2.0,12.000000,5.026641,2.214286,0.675000,6.566667
2,2019-02-08,0,1095.060059,5.805648,-10.267495,53.630592,0.306161,39.197539,-29.663042,-12.035486,...,3.000000,24.000000,16.000000,1.000000,1.0,12.000000,8.259351,2.350000,0.900000,9.300000
3,2019-02-11,0,1095.010010,12.109340,-1.015043,59.919780,-12.517981,36.486913,-34.995952,-12.471680,...,2.000000,22.000000,20.000000,4.000000,0.0,10.000000,9.891429,2.069444,1.600000,8.600000
4,2019-02-12,1,1121.369995,6.390824,-82.130350,211.913261,20.252170,122.859405,-164.221742,-45.784676,...,3.419543,45.266850,134.814683,4.000000,1.0,12.000000,16.288132,10.861708,1.267817,18.083712


In [41]:
feature_without_label_date = feature_with_label_date.iloc[:, 3:]

In [42]:
feature_without_label_date.head()

,0,1,2,3,4,5,6,7,8,9,...,num_pos_VB,num_pos_NN,num_neg_JJ,num_neg_RB,num_neg_VB,num_neg_NN,total_score_JJ,total_score_RB,total_score_VB,total_score_NN
0,-31.065374,26.964167,207.587202,-29.100772,95.531243,-118.985815,-24.375886,138.741649,-99.359529,-59.658105,...,3.998121,55.419917,53.802890,42.127785,0.0,59.779481,23.410036,4.487775,1.999061,9.431500
1,-1.954738,-14.441650,63.646112,-10.731836,39.554150,-40.863590,-10.272327,47.880675,-21.620840,-11.568443,...,2.000000,18.000000,37.000000,8.000000,2.0,12.000000,5.026641,2.214286,0.675000,6.566667
2,5.805648,-10.267495,53.630592,0.306161,39.197539,-29.663042,-12.035486,48.976293,-21.195179,-13.738420,...,3.000000,24.000000,16.000000,1.000000,1.0,12.000000,8.259351,2.350000,0.900000,9.300000
3,12.109340,-1.015043,59.919780,-12.517981,36.486913,-34.995952,-12.471680,36.736582,-16.731461,-14.205209,...,2.000000,22.000000,20.000000,4.000000,0.0,10.000000,9.891429,2.069444,1.600000,8.600000
4,6.390824,-82.130350,211.913261,20.252170,122.859405,-164.221742,-45.784676,166.307355,-66.180203,-45.345804,...,3.419543,45.266850,134.814683,4.000000,1.0,12.000000,16.288132,10.861708,1.267817,18.083712


- adding lagged info

In [54]:
# regenerate feature with time window

historial_feature_columns = [aggregated_vector.columns.to_list()[0]]
for lag_num in range(1, time_window + 1):
    historial_feature_columns += [fea + '_lag' + str(lag_num) for fea in aggregated_vector.columns.to_list()[1:]] 
    
feature_with_lagged_info = pd.DataFrame(columns=historial_feature_columns)

for i in range(len(feature_without_label_date) - 1, time_window - 1, -1):
    sys.stdout.write('\r')
    # the exact output you're looking for:
    sys.stdout.write("Generating: %.02f" % ((len(feature_without_label_date) - 1 - i) / (len(feature_without_label_date) - 1 - time_window) * 100))
    sys.stdout.flush()
    
    # concatenate lagged feature horizontally to add temporal info
    prev_N_days_sum_vector = feature_without_label_date.loc[i - 1].to_numpy()
    for delta in range(2, 1 + time_window):
        prev_N_days_sum_vector = np.concatenate((prev_N_days_sum_vector, 
                                                 feature_without_label_date.loc[i - delta].to_numpy()), 
                                                axis=None)

    # normalize
    # prev_N_days_sum_vector = prev_N_days_sum_vector / np.linalg.norm(prev_N_days_sum_vector)
    
    # add date
    feature_with_date = np.concatenate((np.array([feature_with_label_date.iloc[i, 0]]), prev_N_days_sum_vector))
    new_row_df = pd.DataFrame(feature_with_date).transpose()
    
    # generate column name for lagged feature
    new_row_df.columns = historial_feature_columns
    
    # insert row
    feature_with_lagged_info = pd.concat([new_row_df, feature_with_lagged_info], ignore_index=True)

Generating: 100.00

In [56]:
feature_with_lagged_info.head()

,Date,0_lag1,1_lag1,2_lag1,3_lag1,4_lag1,5_lag1,6_lag1,7_lag1,8_lag1,...,num_pos_VB_lag2,num_pos_NN_lag2,num_neg_JJ_lag2,num_neg_RB_lag2,num_neg_VB_lag2,num_neg_NN_lag2,total_score_JJ_lag2,total_score_RB_lag2,total_score_VB_lag2,total_score_NN_lag2
0,2019-02-08,-1.9547378430000009,-14.441649583200002,63.64611239099997,-10.731835941300002,39.55415032889999,-40.86358978569999,-10.272326656500006,47.88067524670001,-21.6208396538,...,3.9981212885306205,55.419917131976916,53.80288958637382,42.12778481249996,0.0,59.77948079232747,23.410036492091198,4.487774748641578,1.9990606442653103,9.431499582390817
1,2019-02-11,5.805648479300003,-10.267494661,53.630592168999996,0.30616061276000045,39.197539260200024,-29.6630420611,-12.035485623449993,48.976292749400024,-21.195179270000004,...,2.0,18.0,37.0,8.0,2.0,12.0,5.026641414141414,2.214285714285715,0.675,6.566666666666666
2,2019-02-12,12.109339575400007,-1.0150432949599992,59.91977977999998,-12.5179810023,36.486913223,-34.9959524369,-12.471680062899999,36.736581938,-16.731460817000002,...,3.0,24.0,16.0,1.0,1.0,12.0,8.259350649350651,2.35,0.9000000000000002,9.3
3,2019-02-13,6.390823674223785,-82.13035037952076,211.91326138538446,20.25216950397589,122.85940453449659,-164.22174159824536,-45.784676431630146,166.3073545051445,-66.18020295728107,...,2.0,22.0,20.0,4.0,0.0,10.0,9.891428571428571,2.0694444444444446,1.6,8.600000000000001
4,2019-02-14,0.17688897399999778,-9.64450826,68.011613363,-8.693842723800001,35.41786913270001,-39.69494913759999,-15.748714003899995,35.07303002029999,-13.761980877699996,...,3.419542997170286,45.26685004856152,134.81468314587306,4.0,1.0,12.0,16.288132350913813,10.861708167313203,1.2678171988681148,18.08371230270645


- fusion with prev price and volume

In [57]:
fused_feature = stock_price_feature.merge(feature_with_lagged_info, left_on="Date", right_on="Date")

In [58]:
fused_feature.head()

,Date,Label,Adj Close,0_lag1,1_lag1,2_lag1,3_lag1,4_lag1,5_lag1,6_lag1,...,num_pos_VB_lag2,num_pos_NN_lag2,num_neg_JJ_lag2,num_neg_RB_lag2,num_neg_VB_lag2,num_neg_NN_lag2,total_score_JJ_lag2,total_score_RB_lag2,total_score_VB_lag2,total_score_NN_lag2
0,2019-02-08,0,1095.060059,-1.9547378430000009,-14.441649583200002,63.64611239099997,-10.731835941300002,39.55415032889999,-40.86358978569999,-10.272326656500006,...,3.9981212885306205,55.419917131976916,53.80288958637382,42.12778481249996,0.0,59.77948079232747,23.410036492091198,4.487774748641578,1.9990606442653103,9.431499582390817
1,2019-02-11,0,1095.010010,5.805648479300003,-10.267494661,53.630592168999996,0.30616061276000045,39.197539260200024,-29.6630420611,-12.035485623449993,...,2.0,18.0,37.0,8.0,2.0,12.0,5.026641414141414,2.214285714285715,0.675,6.566666666666666
2,2019-02-12,1,1121.369995,12.109339575400007,-1.0150432949599992,59.91977977999998,-12.5179810023,36.486913223,-34.9959524369,-12.471680062899999,...,3.0,24.0,16.0,1.0,1.0,12.0,8.259350649350651,2.35,0.9000000000000002,9.3
3,2019-02-13,0,1120.160034,6.390823674223785,-82.13035037952076,211.91326138538446,20.25216950397589,122.85940453449659,-164.22174159824536,-45.784676431630146,...,2.0,22.0,20.0,4.0,0.0,10.0,9.891428571428571,2.0694444444444446,1.6,8.600000000000001
4,2019-02-14,1,1121.670044,0.17688897399999778,-9.64450826,68.011613363,-8.693842723800001,35.41786913270001,-39.69494913759999,-15.748714003899995,...,3.419542997170286,45.26685004856152,134.81468314587306,4.0,1.0,12.0,16.288132350913813,10.861708167313203,1.2678171988681148,18.08371230270645


- check imbalance

In [59]:
np.count_nonzero(fused_feature.Label.to_list()) / len(fused_feature)

0.549800796812749

In [60]:
fused_feature.head()

,Date,Label,Adj Close,0_lag1,1_lag1,2_lag1,3_lag1,4_lag1,5_lag1,6_lag1,...,num_pos_VB_lag2,num_pos_NN_lag2,num_neg_JJ_lag2,num_neg_RB_lag2,num_neg_VB_lag2,num_neg_NN_lag2,total_score_JJ_lag2,total_score_RB_lag2,total_score_VB_lag2,total_score_NN_lag2
0,2019-02-08,0,1095.060059,-1.9547378430000009,-14.441649583200002,63.64611239099997,-10.731835941300002,39.55415032889999,-40.86358978569999,-10.272326656500006,...,3.9981212885306205,55.419917131976916,53.80288958637382,42.12778481249996,0.0,59.77948079232747,23.410036492091198,4.487774748641578,1.9990606442653103,9.431499582390817
1,2019-02-11,0,1095.010010,5.805648479300003,-10.267494661,53.630592168999996,0.30616061276000045,39.197539260200024,-29.6630420611,-12.035485623449993,...,2.0,18.0,37.0,8.0,2.0,12.0,5.026641414141414,2.214285714285715,0.675,6.566666666666666
2,2019-02-12,1,1121.369995,12.109339575400007,-1.0150432949599992,59.91977977999998,-12.5179810023,36.486913223,-34.9959524369,-12.471680062899999,...,3.0,24.0,16.0,1.0,1.0,12.0,8.259350649350651,2.35,0.9000000000000002,9.3
3,2019-02-13,0,1120.160034,6.390823674223785,-82.13035037952076,211.91326138538446,20.25216950397589,122.85940453449659,-164.22174159824536,-45.784676431630146,...,2.0,22.0,20.0,4.0,0.0,10.0,9.891428571428571,2.0694444444444446,1.6,8.600000000000001
4,2019-02-14,1,1121.670044,0.17688897399999778,-9.64450826,68.011613363,-8.693842723800001,35.41786913270001,-39.69494913759999,-15.748714003899995,...,3.419542997170286,45.26685004856152,134.81468314587306,4.0,1.0,12.0,16.288132350913813,10.861708167313203,1.2678171988681148,18.08371230270645


- import ML lib

In [61]:
# from bert_serving.client import BertClient
import math
from string import punctuation
import matplotlib.pyplot as plt
import re
import os
# from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
# import warnings
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_validate, GridSearchCV
import joblib
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score, roc_auc_score
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.svm import SVC
import xgboost as xgb

# feature selection
from sklearn.feature_selection import SelectKBest, chi2, f_classif

In [62]:
fused_feature.iloc[:, 1:].shape

(251, 1602)

In [63]:
# import seaborn as sns

# # get first 20 dimensions from the 768-dimensional vector
# feature = fused_feature.iloc[:, 1:21]
# corr = feature.corr()
# _ = plt.figure(figsize=(30, 30))
# ax = sns.heatmap(
#     corr, 
#     vmin=-1, vmax=1, center=0,
#     cmap=sns.diverging_palette(20, 220, n=11), # better if the palette color num is odd
#     square=True
# )
# ax.set_xticklabels(
#     ax.get_xticklabels(),
#     rotation=45,
#     horizontalalignment='right'
# );

- feature selection

In [64]:
# apply SelectKBest class to extract top 10 best features
X = fused_feature.iloc[:, 2:]
y = fused_feature.Label
bestfeatures = SelectKBest(score_func=f_classif, k=25)
fit = bestfeatures.fit(X, y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
# concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns, dfscores], axis=1)
featureScores.columns = ['Specs', 'Score']  # naming the dataframe columns
print(featureScores.nlargest(25, 'Score'))  # print 10 best features

                 Specs     Score
737           736_lag1  8.518014
475           474_lag1  7.843476
1185          384_lag2  7.487301
104           103_lag1  6.496562
635           634_lag1  5.890305
696           695_lag1  5.848931
443           442_lag1  5.802122
1070          269_lag2  5.742958
46             45_lag1  5.626910
1243          442_lag2  5.519898
936           135_lag2  5.341093
684           683_lag1  5.311642
905           104_lag2  5.303958
777   num_pos_emo_lag1  5.290085
10              9_lag1  5.282683
683           682_lag1  5.245080
747           746_lag1  5.179303
71             70_lag1  5.109307
651           650_lag1  5.065552
1301          500_lag2  5.012964
583           582_lag1  4.988696
943           142_lag2  4.930583
979           178_lag2  4.920043
1561          760_lag2  4.871108
75             74_lag1  4.718762


/Users/frostace/.conda/envs/bert_env/lib/python3.6/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [ 770  771  774  778 1570 1571 1574 1578] are constant.
  UserWarning)
/Users/frostace/.conda/envs/bert_env/lib/python3.6/site-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [65]:
selected_feature_columns = np.add(2, featureScores.nlargest(110, 'Score').index.to_list())

In [66]:
def string_to_datetime(string):
    date_parts = [int(part) for part in string.split('-')]
    return datetime.date(date_parts[0], date_parts[1], date_parts[2])

In [67]:
fused_feature.Date = fused_feature.Date.apply(lambda x: string_to_datetime(x))
fused_feature.to_csv("final_feature_from_twitter.csv", index=False)

- train test split

In [73]:
X = fused_feature.iloc[:, 2:].to_numpy()
# X = fused_feature.iloc[:, selected_feature_columns].to_numpy()
y = fused_feature.Label.to_list()

In [74]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [75]:
xgb_model = xgb.XGBClassifier(learning_rate=0.03,
                                n_estimators=600,
                                max_depth=7,
                                min_child_weight=5,
                                gamma=0,
                                subsample=0.8,
                                colsample_bytree=0.8,
                                objective='binary:logistic',
                                nthread=4,
                                scale_pos_weight=1,
                                seed=27)
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0,
              learning_rate=0.03, max_delta_step=0, max_depth=7,
              min_child_weight=5, missing=None, n_estimators=600, n_jobs=1,
              nthread=4, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27,
              silent=None, subsample=0.8, verbosity=1)

In [78]:
y_pred = xgb_model.predict(X_train)
y_pred_prob = xgb_model.predict_proba(X_train)[:,1]

In [79]:
print("\nModel Report")
print("Accuracy : %.4g" % accuracy_score(y_train, y_pred))
print("AUC Score (Train): %f" % roc_auc_score(y_train, y_pred_prob))


Model Report
Accuracy : 1
AUC Score (Train): 1.000000


In [76]:
y_pred = xgb_model.predict(X_test)
y_pred_prob = xgb_model.predict_proba(X_test)[:,1]

In [77]:
print("\nModel Report")
print("Accuracy : %.4g" % accuracy_score(y_test, y_pred))
print("AUC Score (Train): %f" % roc_auc_score(y_test, y_pred_prob))


Model Report
Accuracy : 0.4337
AUC Score (Train): 0.499412
